In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import tensorflow as tf
import time

import subprocess
import psutil

In [ ]:
start_time = time.time()

In [ ]:
# Load data
df = pd.read_csv('/kaggle/input/pdc-dataset/pdc_dataset_with_target.csv')

# Check and remove duplicates
df = df.drop_duplicates()


In [ ]:
df.isnull().sum()
df = df.dropna()
df.isnull().sum()

In [ ]:
# Column definitions
numerical_cols = ['feature_1', 'feature_2', 'feature_4', 'feature_6','feature_7']
categorical_cols = ['feature_3', 'feature_5']

In [ ]:
for feature in numerical_cols:
    plt.figure()
    sns.boxplot(x=df[feature])
    plt.title(f'Outlier Detection: {feature}')
    plt.xlabel(feature)
    plt.show()

In [ ]:
# Remove outliers using IQR
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]

In [ ]:
# Split data
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Normalize numeric features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")

In [ ]:
# To store logs
cpu_log, mem_log, gpu_log = [], [], []
# Custom callback to log resource usage
class ResourceLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # CPU and RAM
        cpu = psutil.cpu_percent()
        mem = psutil.virtual_memory().percent

        # GPU: get memory usage from nvidia-smi
        try:
            output = subprocess.check_output(
                ['nvidia-smi', '--query-gpu=utilization.gpu,memory.used',
                 '--format=csv,noheader,nounits'])
            gpu_util, gpu_mem = map(int, output.decode().strip().split(', '))
        except Exception as e:
            gpu_util, gpu_mem = 0, 0  # fallback if no GPU

        cpu_log.append(cpu)
        mem_log.append(mem)
        gpu_log.append(gpu_mem)

        print(f"[Epoch {epoch+1}] CPU: {cpu}%, RAM: {mem}%, GPU Memory Used: {gpu_mem} MB")



# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    # tf.keras.layers.Dense(16, activation='relu'),
    # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

# Train
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1, callbacks=[ResourceLogger()])

# Evaluate
y_pred_probs = model.predict(X_test).flatten()
y_pred = (y_pred_probs > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_probs))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))